**TF-IDF**


In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

In [ ]:
# Загрузка датасета
df = pd.read_csv('PATH_TO_DATASET')

In [ ]:
print(df.head())

In [ ]:
# Разделение данных на признаки и целевую переменную
X = df['utterance']  # Тексты
y = df['label']      # Метки

In [ ]:
# Векторизация текстов с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Ограничим количество признаков для простоты
X_vectorized = vectorizer.fit_transform(X)

In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

In [ ]:
# Обучение логистической регрессии с увеличенным class_weight для эвфемизмов
model = LogisticRegression(class_weight={0: 1, 1: 20}, random_state=42)  # Вес 20 для класса 1
model.fit(X_train, y_train)

In [ ]:
# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

In [ ]:
# F1-score для класса 1 (эвфемизмы)
f1 = f1_score(y_test, y_pred, pos_label=1)
print(f"F1-score для класса 1 (эвфемизмы): {f1:.2f}")

In [ ]:
# Оценка модели
print("Classification Report:")
print(classification_report(y_test, y_pred))

Попробуем уменьшить количество текстов с меткой 0, чтобы сбалансировать датасет, и используем все тексты с меткой 1**

In [ ]:
# Разделение датасета на тексты с меткой 0 и 1
df_label_0 = df[df['label'] == 0]  # Тексты с меткой 0
df_label_1 = df[df['label'] == 1]  # Тексты с меткой 1

In [ ]:
len(df_label_0)

In [ ]:
len(df_label_1)

In [ ]:
# Уменьшение количества текстов с меткой 0
# Текстов с 0 возьмем в 8 раз больше, чем с 1
num_samples = len(df_label_1)  # Количество текстов с меткой 1
df_label_0_sampled = df_label_0.sample(n=num_samples*9, random_state=42)  # Выборка текстов с меткой 0

In [ ]:
# Создание сбалансированного датасета
balanced_df = pd.concat([df_label_0_sampled, df_label_1], ignore_index=True)

In [ ]:
# Проверка баланса классов
print("Распределение классов в сбалансированном датасете:")
print(balanced_df['label'].value_counts())

In [ ]:
# Векторизация текстов с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Ограничиваем количество признаков
X = vectorizer.fit_transform(balanced_df['utterance'])
y = balanced_df['label']
texts = balanced_df['utterance']  # Добавим тексты отдельно, чтобы позже использовать их для ручного анализа

In [ ]:
# Разделение данных на обучающую и тестовую выборки

# X_train, X_test, y_train, y_test = train_test_split(X, balanced_df['label'], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test, texts_train, texts_test = train_test_split(
    X, y, texts, test_size=0.2, random_state=42
)

In [ ]:
# Обучение логистической регрессии
# Увеличиваем вес для редкого класса (эвфемизмы) до 5
model = LogisticRegression(class_weight={0: 1, 1: 5}, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

In [ ]:
# Оценка модели
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# F1-score для класса 1 (эвфемизмы)
f1 = f1_score(y_test, y_pred, pos_label=1)
print(f"F1-score для класса 1 (эвфемизмы): {f1:.2f}")

In [ ]:
# F1-score для класса 0 (не-эвфемизмы)
f1 = f1_score(y_test, y_pred, pos_label=0)
print(f"F1-score для класса 0 (не-эвфемизмы): {f1:.2f}")

In [ ]:
# class weight = 20

# если брать текстов с 0 в 7 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.42
# F1-score для класса 0 (не-эвфемизмы): 0.83

# если брать текстов с 0 в 8 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.42
# F1-score для класса 0 (не-эвфемизмы): 0.86

# если брать текстов с 0 в 9 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.39
# F1-score для класса 0 (не-эвфемизмы): 0.87

# если брать текстов с 0 в 10 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.36
# F1-score для класса 0 (не-эвфемизмы): 0.89

# если брать текстов с 0 в 15 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.30
# F1-score для класса 0 (не-эвфемизмы): 0.92

# если брать текстов с 0 в 20 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.29
# F1-score для класса 0 (не-эвфемизмы): 0.94

In [ ]:
# class weight = 15

# если брать текстов с 0 в 8 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.44
# F1-score для класса 0 (не-эвфемизмы): 0.88

# если брать текстов с 0 в 9 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.40
# F1-score для класса 0 (не-эвфемизмы): 0.88

In [ ]:
# class weight = 10

# если брать текстов с 0 в 8 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.45
# F1-score для класса 1 0 (не-эвфемизмы): 0.90

# если брать текстов с 0 в 9 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.42
# F1-score для класса 0 (не-эвфемизмы): 0.90

In [ ]:
# class weight = 5

# ---------------------------------------------------------------
# если брать текстов с 0 в 8 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.45
# F1-score для класса 0 (не-эвфемизмы): 0.92                                                          ю
# ---------------------------------------------------------------

# если брать текстов с 0 в 9 раз больше, чем с 1, то :
# F1-score для класса 1 (эвфемизмы): 0.44
# F1-score для класса 0 (не-эвфемизмы): 0.93

In [ ]:
# Сохраним предсказания для ручного анализа

# Предсказания
y_pred = model.predict(X_test)
y_probs = model.predict_proba(X_test)[:, 1]

# Оценка
print("Classification Report:")
print(classification_report(y_test, y_pred))

f1_class_1 = f1_score(y_test, y_pred, pos_label=1)
f1_class_0 = f1_score(y_test, y_pred, pos_label=0)
print(f"F1-score для класса 1 (эвфемизмы): {f1_class_1:.2f}")
print(f"F1-score для класса 0 (не-эвфемизмы): {f1_class_0:.2f}")

results_df = pd.DataFrame({
    'utterance': texts_test.values,
    'true_label': y_test.values,
    'predicted_label': y_pred,
    'predicted_prob': y_probs
})

# Ошибки модели
fp = results_df[(results_df['true_label'] == 0) & (results_df['predicted_label'] == 1)]
fn = results_df[(results_df['true_label'] == 1) & (results_df['predicted_label'] == 0)]
errors = pd.concat([fp, fn]).sample(n=min(50, len(fp) + len(fn)), random_state=42)

# Сомнительные предсказания (уверенность между 0.4 и 0.6)
uncertain = results_df[
    (results_df['predicted_prob'] > 0.4) & (results_df['predicted_prob'] < 0.6)
].sample(n=min(30, len(results_df)), random_state=42)

# True Positives
tp = results_df[
    (results_df['true_label'] == 1) & (results_df['predicted_label'] == 1)
].sample(n=min(20, len(results_df)), random_state=42)

# Объединение всех примеров в один датафрейм
final_sample = pd.concat([errors, uncertain, tp]).sample(frac=1, random_state=42)  # перемешаем

final_sample.to_csv("baseline_predictions_to_manual_check.csv", index=False, encoding='utf-8')